In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets # for the widgets
from IPython.display import display # to display them
import pylab
global dimension_to_predict
global char_val
import os

'D:\\SEM-2\\Bio\\fin-project'

In [111]:
def select_variable_to_predict(Variable_to_be_predicted='T'):
  global dimension_to_predict
  global char_val
  char_val = Variable_to_be_predicted
  if Variable_to_be_predicted ==   'T':
    dimension_to_predict = 1
  elif Variable_to_be_predicted == 'I':
    dimension_to_predict = 2
  elif Variable_to_be_predicted == 'V':
    dimension_to_predict = 3
  elif Variable_to_be_predicted == 'J':
    dimension_to_predict = 4
  else:
    print('error in choosing option')
widgets.interact(select_variable_to_predict,Variable_to_be_predicted=['T','I','V','J'])

interactive(children=(Dropdown(description='Variable_to_be_predicted', options=('T', 'I', 'V', 'J'), value='T'…

<function __main__.select_variable_to_predict>

In [114]:
############## To run on remote system  ##################

#model_T = tf.keras.models.load_model('enter your remote path here')
#model_I = tf.keras.models.load_model('enter your remote path here')
#model_V = tf.keras.models.load_model('enter your remote path here')
#model_J = tf.keras.models.load_model('enter your remote path here')
#loadedArr = np.loadtxt(r'enter your remote path here')

################# To be run on host system ###################

model_T = tf.keras.models.load_model(os.path.join(os.path.dirname('log_viewer.ipynb'),'pred_T_log'))
model_T = tf.keras.models.load_model(os.path.join(os.path.dirname('log_viewer.ipynb'),'pred_I_log'))
model_T = tf.keras.models.load_model(os.path.join(os.path.dirname('log_viewer.ipynb'),'pred_V_log'))
model_T = tf.keras.models.load_model(os.path.join(os.path.dirname('log_viewer.ipynb'),'pred_J_log'))
loadedArr = np.loadtxt(os.path.join(os.path.dirname('log_viewer.ipynb'),'S_1_point_0.csv'))


def execute(dimension_to_predict,char_val):
  print(char_val)
  print('inside')
  print(dimension_to_predict)
  print(char_val)
  #loadedArr = np.loadtxt('drive/My Drive/Colab Notebooks/S_1_point_1.csv')
  loadedOriginal = loadedArr.reshape(loadedArr.shape[0], loadedArr.shape[1] // 10, 10)
  #print(dimension_to_predict)
  a = loadedOriginal
  a[:,:,1:5][a[:,:,1:5]<0] = 1e-12
  a[:,:,1:5][a[:,:,1:5]==0] = 1e-7
  a[:,:,1:5] = np.log(a[:,:,1:5])
  simulations = a.shape[0]
  timesteps = a.shape[1]
  dimensions = a.shape[2]
  no_of_sims_for_testing = 100
  a_test = a[:no_of_sims_for_testing]
  a_train = a[no_of_sims_for_testing:]
  batches_of_TrainX = []
  for i in range(a_train.shape[0]):
          data = {
          'time_steps' : a_train[i,:,0], 
          '[T]' : a_train[i,:,1], 
          '[I]' : a_train[i,:,2],
          '[V]' : a_train[i,:,3],
          '[J]' : a_train[i,:,4] ,
                }
          data = pd.DataFrame(data) 
          batches_of_TrainX.append(data)
  batches_of_TrainX = np.array(batches_of_TrainX)
  from sklearn.compose import ColumnTransformer
  import sklearn
  scaled_batches = batches_of_TrainX.reshape(batches_of_TrainX.shape[0]*batches_of_TrainX.shape[1],batches_of_TrainX.shape[2])
  scaler_1 = sklearn.preprocessing.MinMaxScaler()
  scaler_1 = scaler_1.fit(scaled_batches)
  scaler_2 = sklearn.preprocessing.MinMaxScaler()
  scaler_2 = scaler_2.fit(scaled_batches[:,dimension_to_predict].reshape(-1,1))
  scalar_3 = sklearn.preprocessing.MinMaxScaler()
  scaler_3 = scaler_1.fit(scaled_batches[:,:5])
  scaled_batches = scaler_1.transform(scaled_batches)
  trainX = []
  for i in range(batches_of_TrainX.shape[0]):
      trainX.append(scaled_batches[i*batches_of_TrainX.shape[1]:(i+1)*batches_of_TrainX.shape[1]])  
  trainX = np.array(trainX)
  batches_of_TrainX_array = trainX

  TrainX = []
  TrainY = []
  n_future = 1  
  n_past = 3 
  window_end_number = timesteps - n_past
  for i in range(batches_of_TrainX_array.shape[0]):
      train_batch = []
      train_tempY = []
      for j in range(n_past, batches_of_TrainX_array.shape[1] - n_future +1):
          train_batch.append(batches_of_TrainX_array[i,j-n_past:j,:])
          train_tempY.append(batches_of_TrainX_array[i, j+n_future-1 : j+n_future , dimension_to_predict])
      TrainX.append(train_batch)
      TrainY.append(train_tempY)
  TrainX = np.array(TrainX)
  TrainY = np.array(TrainY)

  TrainX = TrainX.reshape(TrainX.shape[0]*TrainX.shape[1],TrainX.shape[2],TrainX.shape[3])
  TrainY = TrainY.reshape(TrainY.shape[0]*TrainY.shape[1],TrainY.shape[2])

  if dimension_to_predict == 1:
    model = model_T
  if dimension_to_predict == 2:
    model = model_I
  if dimension_to_predict == 3:
    model = model_V
  if dimension_to_predict == 4:
    model = model_J

  batches_of_TestX = []
  for i in range(a_test.shape[0]):
          data = {
          'time_steps' : a_test[i,:,0], 
          '[T]' : a_test[i,:,1], 
          '[I]' : a_test[i,:,2],
          '[V]' : a_test[i,:,3],
          '[J]' : a_test[i,:,4] ,
                }
          data = pd.DataFrame(data) 
          batches_of_TestX.append(data)
  batches_of_TestX = np.array(batches_of_TestX)
  scaled_batches_1 = batches_of_TestX.reshape(batches_of_TestX.shape[0]*batches_of_TestX.shape[1],batches_of_TestX.shape[2])
  scaled_batches_1 = scaler_1.transform(scaled_batches_1)
  testX = []
  for i in range(batches_of_TestX.shape[0]):
      testX.append(scaled_batches[i*batches_of_TestX.shape[1]:(i+1)*batches_of_TestX.shape[1]])  
  testX = np.array(testX)
  TestX = []
  for i in range(testX.shape[0]):
      test_batch = []
      for j in range(n_past, testX.shape[1] - n_future +1):
          test_batch.append(testX[i,j-n_past:j,:])
      TestX.append(test_batch)
  TestX = np.array(TestX)
  TestX = []
  for i in range(testX.shape[0]):
      test_batch = []
      for j in range(n_past, testX.shape[1] - n_future +1):
          test_batch.append(testX[i,j-n_past:j,:])
      TestX.append(test_batch)
  TestX = np.array(TestX)
  TestX = TestX.reshape(TestX.shape[0]*TestX.shape[1],TestX.shape[2],TestX.shape[3])
  Y_pred_V = model.predict(TestX,batch_size=window_end_number)
  Y_pred_V = Y_pred_V.reshape(-1)
  Y_pred_V_scaled_back = scaler_2.inverse_transform(Y_pred_V.reshape(-1,1))
  fin_values = []
  for i in range(100):
      fin_values.append(Y_pred_V_scaled_back[i*window_end_number:(i+1)*window_end_number])  
  fin_values = np.array(fin_values)
  fin_values = fin_values.reshape(100,window_end_number)
  original_vals = a_test[:,:,:]
  original_vals.shape
  original_vals = original_vals[:,timesteps-window_end_number:,dimension_to_predict]

  def View_results(value=0,end_value=0):
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    ax.set_xlabel('Time')
    ax.set_ylabel('Value of ' + char_val)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Value of all variables')
    ax.set_title("Prediction vs True value of " + char_val)
    ax.plot(np.exp(fin_values[value][:end_value]),c='red',label='prediction')
    ax.plot(np.exp(original_vals[value][:end_value]),c='green',label='Groundtruth')
    ax.legend()
    ax2.set_title("SIR model")
    plt.plot(np.exp(batches_of_TestX[value][:end_value]))
    plt.show()
  widgets.interact(View_results,value=widgets.IntSlider(
    value=0,
    min=0,
    max=no_of_sims_for_testing-1,
    step=1,
    description='TestSimNumber'),end_value=widgets.IntSlider(
    value=5,
    min=0,
    max=timesteps,
    step=1,
    description='X_lim_of_simulation'));

In [115]:
execute(dimension_to_predict,char_val)

J
inside
4
J


interactive(children=(IntSlider(value=0, description='TestSimNumber', max=99), IntSlider(value=5, description=…